Мы добилиись f1 score на стандартной логистической регрессии: 0.67

Примечание: Паралельно мы ведём разработку упомянутого ранее модуля, который модифицирует подготовленные данные, так чтобы мы знали не версию, а количество лет.
Данный модуль пока не дописан, так что далее мы будем писать без данных полей.

## CatBoost

Сейчас мы собираемся применить альтернативный подход. Мы помним рекомендацию преподавателя воспользоваться CatBoost. Один из нас посмотрел https://www.youtube.com/watch?v=UYDwhuyWYSo 
Отметим, что это библиотека, которая использует градиентный бустинг. Проверим, сможем ли мы получить результат выше чем 0.67.

Для начала подключаем все необходимые материалы, как нам указали в отзыве на промежуточной версии.

In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split

Теперь можно открыть файлы, которые мы подготовили ранее

In [2]:
train_df_csv = pd.read_csv('../data/train_processed.csv',delimiter = ';')
test_df_csv = pd.read_csv('../data/test_processed.csv',delimiter = ';')

In [3]:
train_df_csv

,request_ts,user_id,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,browser,browser_version,os,os_version,target
0,1701011363,fb858e8e0a2bec074450eaf94b627fd3,c31b4e,470e75,f6155e,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,119.0.0,Android,10,0
1,1700986581,46a5f128fd569c764a92c2eaa788095e,c31b4e,44520b,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,111.0.0,Android,10,0
2,1701011071,5a74e9ac53ffb21a20cce117c0ad77ba,c31b4e,616bb9,af47f1,12498,2451,10304,-6380,11608,3106,-2188,10573,3347,21870,Yandex Browser,20.12.5,Android,11,0
3,1700992803,af735816ca19115431ae3d89518c8c91,c31b4e,3c9dca,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,119.0.0,Android,10,0
4,1701021666,364f0ae0a3f29a685c4fb5bae6033b9a,c31b4e,776e76,10b7947,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Yandex Browser,18.11.1,Android,4.4.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601285,1701009148,11516096d9f97463424523e6154d5255,c31b4e,8ccc01,e56e80,18174,17591,3257,2581,8043,13263,2312,1791,9121,5170,Yandex Browser,23.11.0,Android,13,1
601286,1700954940,d078ecb7c7e9eaf65189fb3d1bed7871,c31b4e,52b73e,e56e80,14961,13188,3978,1812,13751,14055,1141,-10698,7083,9270,Chrome,119.0.0,Windows,10,0
601287,1700994910,3543d64627ead3a519e3199834e2a148,c31b4e,33ed7a,10b7947,12332,17481,12664,-984,5146,11562,4316,15017,1423,-981,Chrome Mobile,105.0.0,Android,11,0
601288,1700989030,37df5ff1d739f61d442b164db6281e46,c31b4e,97023a,e56e80,13459,-6413,13160,13187,5498,3875,13246,10259,-1630,-14647,Chrome Mobile,94.0.4606,Android,11,0


Теперь разделим на то, что мы ищем, то есть исключительно на колонку target. И на то, что может быть использовано для обучения. (Мы будем игнорировать user_id, browser_version и os_version)

In [4]:
X = train_df_csv.drop(columns=['target', 'user_id','browser_version','os_version'])
y = train_df_csv['target']

Разделим данные на обучающущую и тестовую выборку

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

Отдельно отметим наличие категориальных признаков, это country_id, region_id, timezone_id, os, browser.

In [6]:
cat_features = ['country_id', 'region_id', 'timezone_id', 'os', 'browser']

Как выяснилось библиотека не приемлет наличие всяких около наловых значений в категориальных признаках, так что уберём всякие NaN, None и прочее. Теперь это отдельная категория - Other.

In [15]:
X_train.isnull()

,request_ts,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,browser,os
385201,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
483415,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
551633,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
328803,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
265436,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
259178,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
365838,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
131932,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [16]:
X_train[cat_features] = X_train[cat_features].replace({None: 'Other', '': 'Other'})
X_train[cat_features] = X_train[cat_features].fillna('Other')

Создаём пул

In [19]:
train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
#test_pool = Pool(data=X_test, label=y_test, cat_features=cat_features)

Теперь инициализируем модель. Параметры взял из примера, возможно, они нам не понравятся и мы пойдём их менять далее.

In [20]:
model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, verbose=100)

Теперь происходит обучение

In [21]:
model.fit(train_pool)

0:	learn: 0.6680155	total: 662ms	remaining: 11m 1s
100:	learn: 0.4820636	total: 51.3s	remaining: 7m 36s
200:	learn: 0.4616178	total: 1m 40s	remaining: 6m 37s
300:	learn: 0.4510519	total: 2m 28s	remaining: 5m 45s
400:	learn: 0.4442387	total: 3m 17s	remaining: 4m 55s
500:	learn: 0.4392912	total: 4m 4s	remaining: 4m 3s
600:	learn: 0.4352641	total: 4m 51s	remaining: 3m 13s
700:	learn: 0.4318392	total: 5m 45s	remaining: 2m 27s
800:	learn: 0.4289044	total: 6m 34s	remaining: 1m 37s
900:	learn: 0.4265132	total: 7m 30s	remaining: 49.5s
999:	learn: 0.4242264	total: 8m 15s	remaining: 0us


In [26]:
X_test[cat_features] = X_test[cat_features].replace({None: 'Other', '': 'Other'})
X_test[cat_features] = X_test[cat_features].fillna('Other')

Проверим точность

In [27]:
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.82


0.82 это хорошо, но это стоит проверить на f1 score

In [28]:
from sklearn.metrics import f1_score 

def GetPrediction(model, X, y, metric = f1_score, **kwargs):
    
    prediction = model.predict(X)
    return metric(y, prediction, **kwargs)

In [29]:
f" f1 на тренировочной выборке: {GetPrediction(model, X_train, y_train):.2f}" +'\n'+f" f1 на валидационной выборке: {GetPrediction(model, X_test, y_test):.2f}"

' f1 на тренировочной выборке: 0.82\n f1 на валидационной выборке: 0.81'